In [1]:
# Read data and draw some figures
import json

# Open and read the JSON file
with open('SLD.json', 'r') as file:
    data = json.load(file)

In [3]:
import networkx as nx


# Create a graph
G = nx.DiGraph()

# Create nodes
counter = 0
for element in data:
    #print(element)
    G.add_nodes_from([(counter, element)])
    counter += 1

#G.add_nodes_from(data)

#print(G.nodes)

donor_recipient_compatibility = {'O': ['O', 'A', 'B', 'AB'], 'A': ['A', 'AB'], 'B':['B', 'AB'], 'AB':['AB']}

# Create edges
for node in G.nodes:
    donors = G.nodes[node]['Donor']
    #print(donors)
    for donor in donors:
        for vertex in G.nodes:
            if node == vertex: continue
            recipient = G.nodes[vertex]['Recipient']
            if recipient not in donor_recipient_compatibility[donor]: continue
            G.add_edge(node, vertex)    
                
print('edges:', G.edges)    

edges: [(1, 3), (1, 6), (1, 20), (1, 24), (1, 25), (1, 28), (1, 31), (1, 34), (1, 37), (1, 49), (1, 50), (1, 57), (1, 59), (1, 62), (1, 64), (1, 65), (1, 68), (1, 69), (1, 77), (1, 79), (1, 86), (1, 87), (1, 88), (1, 97), (1, 98), (1, 100), (1, 102), (1, 107), (1, 108), (1, 111), (1, 116), (1, 119), (1, 120), (1, 127), (1, 128), (1, 143), (1, 151), (1, 153), (1, 159), (1, 160), (1, 162), (1, 163), (1, 165), (1, 166), (1, 172), (1, 174), (1, 178), (1, 184), (1, 189), (1, 193), (1, 194), (1, 197), (1, 198), (1, 204), (1, 205), (1, 207), (1, 209), (1, 213), (1, 214), (1, 221), (1, 223), (1, 229), (1, 230), (1, 231), (1, 235), (1, 236), (1, 238), (1, 250), (1, 253), (1, 254), (1, 257), (1, 259), (1, 263), (1, 281), (1, 285), (1, 292), (1, 294), (1, 300), (1, 302), (1, 303), (1, 304), (1, 306), (1, 307), (1, 308), (1, 309), (1, 312), (1, 313), (1, 319), (1, 320), (1, 321), (1, 322), (1, 326), (1, 331), (1, 336), (1, 337), (1, 341), (1, 344), (1, 347), (1, 348), (1, 362), (1, 363), (1, 367),

In [4]:
# Find cycles of length 3

potential_cycle_3 = []

H = G.to_undirected()

for (u,v) in H.edges:
    for k in nx.common_neighbors(H, u, v):
        potential_cycle_3.append((u,v,k))
        
#print("potential_cycle_3:", potential_cycle_3)        
        
cycle_3 = [] 

for (u,v,k) in potential_cycle_3:
    if (u,v) in G.edges and (v,k) in G.edges and (k,u) in G.edges:
        cycle_3.append((u,v,k,u))
    if (v,u) in G.edges and (u,k) in G.edges and (k,v) in G.edges: 
        cycle_3.append((v,u,k,v))

print("cycle_3:", cycle_3) 

cycle_3: []


In [5]:
# Find cycles of length 2

cycle_2 = []

for (i,j) in G.edges:
    G.edges[(i,j)]["visited"] = False

for (i,j) in G.edges:
    if G.edges[(i,j)]["visited"] == True: continue
    if (j,i) in G.edges:
        cycle_2.append((i,j,i))
        G.edges[(j,i)]["visited"] = True
            
print("cycles of length 2:", cycle_2)

cycles of length 2: [(1, 3, 1), (1, 20, 1), (1, 24, 1), (1, 25, 1), (1, 31, 1), (1, 37, 1), (1, 49, 1), (1, 50, 1), (1, 57, 1), (1, 59, 1), (1, 62, 1), (1, 64, 1), (1, 65, 1), (1, 68, 1), (1, 69, 1), (1, 77, 1), (1, 79, 1), (1, 86, 1), (1, 87, 1), (1, 88, 1), (1, 97, 1), (1, 98, 1), (1, 102, 1), (1, 107, 1), (1, 111, 1), (1, 116, 1), (1, 120, 1), (1, 127, 1), (1, 128, 1), (1, 143, 1), (1, 151, 1), (1, 153, 1), (1, 159, 1), (1, 160, 1), (1, 162, 1), (1, 163, 1), (1, 165, 1), (1, 166, 1), (1, 172, 1), (1, 174, 1), (1, 178, 1), (1, 184, 1), (1, 189, 1), (1, 193, 1), (1, 194, 1), (1, 197, 1), (1, 198, 1), (1, 205, 1), (1, 207, 1), (1, 209, 1), (1, 221, 1), (1, 229, 1), (1, 230, 1), (1, 235, 1), (1, 238, 1), (1, 250, 1), (1, 253, 1), (1, 254, 1), (1, 259, 1), (1, 263, 1), (1, 281, 1), (1, 285, 1), (1, 292, 1), (1, 294, 1), (1, 300, 1), (1, 302, 1), (1, 306, 1), (1, 309, 1), (1, 313, 1), (1, 319, 1), (1, 320, 1), (1, 321, 1), (1, 322, 1), (1, 326, 1), (1, 331, 1), (1, 336, 1), (1, 337, 1), (

In [6]:
# Let's find a maximum matching
import gurobipy as gp
from gurobipy import GRB

# Create model object
m = gp.Model()

# Create variable for each edge
x = m.addVars(G.edges, vtype=GRB.BINARY)

# Objective function: maximize number of edges
m.setObjective(gp.quicksum(x[e] for e in G.edges), GRB.MAXIMIZE)

# The number of incomming arcs to each vertex is at most one
m.addConstrs(gp.quicksum(x[e] for e in G.edges(v)) <= 1 for v in G.nodes)

# The number of incomming arcs should be equal to the number of outgoing arcs
#m.addConstrs(gp.quicksum(x[(u,v)] for u in G.neighbors(v) if (u,v) in G.edges) == gp.quicksum(x[(v,u)] for u in G.neighbors(v) if (v,u) in G.edges) for v in G.nodes)

# Solve
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-23
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 23.3.0 23D60)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 725 rows, 177197 columns and 177197 nonzeros
Model fingerprint: 0x76fc0fd6
Variable types: 0 continuous, 177197 integer (177197 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 620.0000000
Presolve removed 725 rows and 177197 columns
Presolve time: 0.09s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.15 seconds (0.04 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 620 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.200000000000e